Import Statements and Set Seed

In [ ]:
import tensorflow as tf
import random
from tensorflow import keras
import numpy as np
from collections import deque
from keras.layers import Conv1D, Dense, Flatten, Activation, MaxPooling1D, Dropout
from math import sin, cos, pi
from numpy.random import randint

from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

from tqdm import tqdm

np.random.seed(295)
random.seed(295)

#CONSTANTS
SIN5 = sin(5*pi/180)
COS5 = cos(5*pi/180)

# Environment settings
EPISODES = 75_000

#episodic batch size
BATCH_SIZE = 128

# Exploration settings
epsilon = 0.96
EPSILON_DECAY = 0.999
MIN_EPSILON = 0.01

#Discount factor
DISCOUNT = 0.98

#target update value
COUNTER_MAX = 5

Memory Class

In [ ]:
class ReplayMemory:
    
    def __init__(self):

        self.alog = np.array([0,0,0]) #log for action
        self.rlog = [] #log for reward
        self.slog = 0 #log for success
        self.sslog = [] #log for success per step
        self.clog = 0 #log for collision
        self.cclog = [] #log for collisions per step
        self.elog = 0 #log for number of episodes
        self.llog = [] #log for the episodic loss

        self.reset()
    
    def reset(self):
        #environment data
        self.p = np.array([])
        self.t = np.array([])

        self.px = np.array([])
        self.py = np.array([])

Environment Class

In [ ]:
class BatchEnvironment:
    SIZE = 600
    
    MOVE_REWARD = -1
    CAPTURE_REWARD = 25
    COLLISION_REWARD = -8

    MAX_EPISODE_STEP = 300

    def __init__(self, BATCH_SIZE):
        self.BATCH_SIZE = BATCH_SIZE

        self.rm = ReplayMemory()

        self.episode_step = np.ones(self.BATCH_SIZE)*self.MAX_EPISODE_STEP
    
    def reset(self):
        #clear the replay buffer
        self.rm.reset()

        #generate new random states
        self.rm.p = randint(0,self.SIZE,(self.BATCH_SIZE,2))*1.0
        self.rm.t = randint(0,self.SIZE,(self.BATCH_SIZE,2))*1.0

        #genrating random theta for vx, vy
        th = 2*np.pi*np.random.random(self.BATCH_SIZE)
        self.rm.px = (0.9*self.SIZE/200)*np.cos(th)
        self.rm.py = (0.9*self.SIZE/200)*np.sin(th)

        #reset episode step
        self.episode_step = np.ones(self.BATCH_SIZE)*self.MAX_EPISODE_STEP
    
    def getCS(self):
        return np.concatenate(((self.rm.p-self.rm.t)/self.SIZE,self.rm.p/self.SIZE,self.rm.px[None].T/6,self.rm.py[None].T/6), axis=1)
    
    def step(self, action):
        #create a current state for training
        cs = self.getCS()

        #update positions first
        self.rm.p[:,0] += self.rm.px
        self.rm.p[:,1] += self.rm.py

        #update the velocity
        temp = self.rm.px.copy()
        self.rm.px = self.rm.px*COS5 - self.rm.py*action*SIN5
        self.rm.py = self.rm.py*COS5 + temp*action*SIN5

        #update the step counter
        self.episode_step -= 1

        #create a new state for training
        ns = self.getCS()

        #check for terminal states
        capture = np.sum(np.abs(self.rm.p-self.rm.t), axis=1)<self.SIZE/20
        self.rm.slog += capture.sum()
        self.episode_step[capture] = 0

        outabound = np.logical_or((self.rm.p>self.SIZE),(self.rm.p<0))
        collision = np.logical_or(outabound[:,0],outabound[:,1])
        self.rm.clog += collision.sum()

        self.episode_step[collision] = 0

        done = (self.episode_step==0)
        s = done.sum()
        self.rm.elog += s

        if s:
            #generate new states for terminal ones
            self.rm.p[done] = randint(0,self.SIZE,(s,2))*1.0
            self.rm.t[done] = randint(0,self.SIZE,(s,2))*1.0

            th = 2*np.pi*np.random.random(s)
            self.rm.px[done] = (0.9*self.SIZE/200)*np.cos(th)
            self.rm.py[done] = (0.9*self.SIZE/200)*np.sin(th)

            #reset episode_step
            self.episode_step[done] = self.MAX_EPISODE_STEP

        #calculate rewards
        reward = np.ones(self.BATCH_SIZE)*self.MOVE_REWARD + capture*self.CAPTURE_REWARD + collision*self.COLLISION_REWARD

        self.rm.rlog.append(reward.mean())

        #return cs, ns, reward, done
        return cs, ns, reward, done

Deep Q-Learning Network

In [ ]:
class DQN:
    def __init__(self):

        # Main model
        self.model = self.create_model()

        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())

        self.update_counter = 0

    def create_model(self):
        model = keras.Sequential()

        model.add(keras.Input((6,1)))

        model.add(Conv1D(128, 3, padding='same'))
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, padding='same'))

        model.add(Conv1D(128, 5, padding='same'))
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, padding='same'))

        model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
        model.add(Dense(32))

        model.add(Dense(3, activation='linear'))  # ACTION_SPACE_SIZE = how many choices (9)
        model.compile(loss="mse", optimizer='adam', metrics=['accuracy'])
        return model

    def train(self, cs, qcs, ns, action, reward, done):
        #query target value for future max rewards
        qns = self.target_model.predict(ns)

        #get the maximum future reward
        max_qns = np.max(qns, axis=1)

        #the discounted maximum future reward for non terminal state
        new_qns = reward + DISCOUNT*max_qns*np.invert(done)

        #create a mask from action space
        mask = np.concatenate((action==-1,action==0,action==1)).reshape((3,action.size)).T

        #set get the true predictions
        fit_qns = np.copy(qcs)
        fit_qns[mask] = new_qns

        loss = (qcs-fit_qns).mean()
        mask = np.random.choice(cs.shape[0],64,replace=False)

        #fit the model
        self.model.fit(cs[mask], fit_qns[mask], batch_size=64, verbose=0, shuffle=True)

        #update the target network
        if done.sum():
            self.update_counter += 1
            
        if self.update_counter == COUNTER_MAX:
            self.target_model.set_weights(self.model.get_weights())
            self.update_counter=0
        
        return loss

    
    #return the q value
    def get_qs(self, state):
        return self.model.predict(state)

Training Loop

In [ ]:
network = DQN()

env = BatchEnvironment(BATCH_SIZE)
env.reset()

#network.model = tf.keras.models.load_model('/content/drive/MyDrive/model2_alt')
#network.target_model.set_weights(network.model.get_weights())
 


for episode in tqdm(range(1, EPISODES + 1), ascii=True, unit='step'):
    #get initial Q values
    Qcs = network.get_qs(env.getCS())

    #get action from q values
    action = np.argmax(Qcs,axis=1)-1
    env.rm.alog += [(action==-1).sum(),(action==0).sum(),(action==1).sum()]

    #epsilon randomization
    r_action = randint(-1,2,BATCH_SIZE)
    mask = np.random.rand(BATCH_SIZE)<epsilon

    action[mask] = r_action[mask]
    epsilon *= EPSILON_DECAY
    epsilon = max(epsilon,MIN_EPSILON)

    #run a step the batch
    cs, ns, reward, done = env.step(action)
    if episode%50 == 0:
        env.rm.sslog.append(env.rm.slog)
        env.rm.cclog.append(env.rm.clog)
        env.rm.slog = 0
        env.rm.clog = 0

    if(episode%1000 == 0):
          network.model.save('/content/drive/MyDrive/model2_alt')
          print('Model saved - ' + str(epsilon))
          episode += 1


    #finally train the model
    loss = network.train(cs, Qcs, ns, action, reward, done)
    env.rm.llog.append(np.abs(loss))

  1%|1         | 999/75000 [02:18<2:24:02,  8.56step/s]

INFO:tensorflow:Assets written to: /content/drive/MyDrive/model2_alt/assets


  1%|1         | 1000/75000 [02:19<9:06:02,  2.26step/s]

Model saved - 0.3529876077801247


  3%|2         | 1999/75000 [04:24<2:32:44,  7.97step/s]

...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/model2_alt/assets


 11%|#         | 8000/75000 [17:03<6:29:31,  2.87step/s]

Model saved - 0.01


 12%|#1        | 8695/75000 [18:31<2:21:15,  7.82step/s]


KeyboardInterrupt: ignored

Saving the model

In [ ]:
network.model.save('/model1')

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 10)

In [ ]:
plt.plot(range(len(env.rm.llog)),env.rm.llog)
plt.xlabel("Loss over steps")
plt.show()

In [ ]:
plt.plot(range(len(env.rm.rlog)),env.rm.rlog)
plt.xlabel("Reward over steps")
plt.show()

In [ ]:
print(env.rm.alog) #actions

In [ ]:
plt.plot(range(len(env.rm.cclog)),env.rm.cclog)
plt.xlabel("Collision over steps")
plt.show()

In [ ]:
plt.plot(range(len(env.rm.sslog)),env.rm.sslog)
plt.xlabel("captures over steps")
plt.show()

In [ ]:
print(5116*256.0/env.rm.elog)

In [ ]:
np.sum(env.rm.sslog)

In [ ]:
np.sum(env.rm.cclog)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')